In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import os
import requests
from bs4 import BeautifulSoup

# Step 1: Get the list of NYSE tickers
def get_nyse_tickers():
    url = 'https://www.nasdaq.com/market-activity/stocks/screener'
    response = requests.get(url)
    print(response)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'class': 'market-activity__table'})
    
    tickers = []
    for row in table.find_all('tr')[1:]:
        ticker = row.find_all('td')[0].text.strip()
        tickers.append(ticker)
        
    return tickers

# Step 2: Download historical data
def download_stock_data(tickers, start_date, end_date, data_folder="stock_data"):
    for ticker in tickers:
        try:
            stock_data = yf.download(ticker, start=start_date, end=end_date)
            if not stock_data.empty:
                file_path = os.path.join(data_folder, f"{ticker}.csv")
                stock_data.to_csv(file_path)
                print(f"Downloaded data for {ticker}")
            else:
                print(f"No data for {ticker}")
        except Exception as e:
            print(f"Could not download data for {ticker}: {e}")

# Create a folder to store the data if it doesn't exist
data_folder = "stock_data"
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

# Define the date range
end_date = datetime.today().date()
start_date = end_date - timedelta(days=3*365)

# Get the list of NYSE tickers
nyse_tickers = get_nyse_tickers()

# Download the stock data
download_stock_data(nyse_tickers, start_date, end_date)
